# Redes Neurais e sua Implementação

In [10]:
import string
import re
import nltk
import pandas as pd
from unidecode import unidecode
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ruasgar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Pré-Processamento básico

In [38]:
df = pd.read_csv("../data/final_dataset.csv")

In [2]:
df.columns

Index(['sentence', 'news_link', 'outlet', 'topic', 'type', 'group_id',
       'num_sent', 'Label_bias', 'Label_opinion', 'article', 'biased_words4',
       'full_article'],
      dtype='object')

- A priori, vamos considerar como tokens todas as palavras que:
    - tenham mais do que 3 letras
    - não possuam números
    - não estejam nas stopwords
    
- Vamos remover os acentos e, mediante a escolha do usuário, aplicar um stemmer.
- Visando o mapeamento das palavras para números inteiros, vamos utilizar o padrão de marcar alguns tokens especiais:
    - 0: padding (completar o tamanho das representações)
    - 1: Start-of-Sequence
    - 2: tokens desconhecidos, necessários para novas palavras

In [39]:
digit_pattern = r'\d+(\.\d+)?'
solo_quotations_pattern = pattern = r"^(?:' '|\\" "|\`\`)$"

remove_quotation = r"'\b|\b'\s|\s'\b|\"\b|\b\"\s|\s\"\b|``\b|\b``\s|\s``\b" # como em "America is Great"
remove_symbols_only = r'\b[^\w\s]+\b' # como em '--'
remove_symbols_if_aside = r'\b[^\w\s]|[^\w\s]\b' # como em 'dog-'

remove_patterns = remove_quotation,remove_symbols_if_aside, remove_symbols_only

stop_words = set(stopwords.words('english')) # talvez o set deixe mais rápido

def preprocess_sentences(df):
    
    prepped_sentences = []

    for s in df['sentence']:
        # Remove acentos e põe tudo para lower case
        s = unidecode(s).lower()
        tokens = word_tokenize(s,language="english")

        tokens = [
            re.sub('|'.join(remove_patterns),'',t)
            for t in tokens 
            if not bool(re.search(digit_pattern,t))
            and t not in string.punctuation
            and t not in stop_words
        ]

        tokens = [
            t for t in tokens
            if not bool(re.match(solo_quotations_pattern,t)) 
            and len(t) >= 3
        ]
        
        prepped_sentences.append(tokens)

    df['sentence'] = prepped_sentences

    return df 


df = preprocess_sentences(df)
df['sentence'].head()


0    [youtube, making, clear, birtherism, platform,...
1    [increasingly, bitter, dispute, american, wome...
2    [may, humanitarian, crisis, driving, vulnerabl...
3    [professor, teaches, climate, change, classes,...
4    [world, antidoping, agency, tuesday, said, rus...
Name: sentence, dtype: object